# Value Iteration & Policy Iteration
> 1. v(s) and policy  for value iteration 演算法對應
2. show animation
2. option ==> do policy iteration

# 1. Policy Iteration

In [15]:
import grid_world
from grid_world import *
import numpy as np

## 第一步：將grid設定成standard狀態，再將可走動的格子reward全設成-0.1    

#### standard grid參數如下：    
g = Grid(3, 4, (2, 0))     
rewards = {(0, 3): 1, (1, 3): -1}    
actions = {    
    (0, 0): ('D', 'R'), (0, 1): ('L', 'R'), (0, 2): ('L', 'D', 'R'),     
    (1, 0): ('U', 'D'), (1, 2): ('U', 'D', 'R'),    
    (2, 0): ('U', 'R'), (2, 1): ('L', 'R'), (2, 2): ('L', 'R', 'U'), (2, 3): ('L', 'U'),    
}

In [16]:
step_cost = -0.1
grid = standard_grid()
grid.rewards.update({
    (0, 0): step_cost,
    (0, 1): step_cost,
    (0, 2): step_cost,
    (1, 0): step_cost,
    (1, 2): step_cost,
    (2, 0): step_cost,
    (2, 1): step_cost,
    (2, 2): step_cost,
    (2, 3): step_cost,
})
states = grid.all_states()
print("states:")
print(states)

print("\nrewards:")
print_values(grid.rewards, grid)

states:
{(0, 0), (1, 3), (2, 1), (2, 3), (1, 0), (0, 3), (0, 1), (1, 2), (2, 0), (2, 2), (0, 2)}

rewards:
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


## 隨機選擇各格子要前進的方向

In [17]:
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
print("policy:")
print_policy(policy, grid)

policy:
---------------------------
  U  |  U  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  D  |  R  |  R  |


## 第二步：初始化與Main function，Policy Evaluation + Policy Improvement

In [18]:
#initialize V, gamma, policy, grid
v = {}
gamma = 0.9
small_enough = 1e-3
for s in states:
    v[s] = 0
    
#前段跑Policy Evaluation，後段跑Policy Improvement
while True:
    while True:
        biggest_change = 0.0
        for s in states:                     #s會得到(0, 1)、(1, 2)...等11種state
            old_v = v[s]
            if s in policy:
                a = policy[s]                #a會得到L,R,D或U
                grid.set_state(s)            #i,j會變成s所在的格子
                r = grid.move(a)             #i,j的數字會隨前進方向改變，r會收到回傳的reward值
                v[s] = r + gamma * v[grid.current_state()]
            biggest_change = max(biggest_change, np.abs(old_v - v[s]))
        if biggest_change < small_enough:
            break 

    is_policy_converged = True
    for s in states:
        if s in policy:
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')
            # loop through all possible actions to find the best current action
            for a in ALL_POSSIBLE_ACTIONS:   #a will loop through L, R, D, and U
                grid.set_state(s)
                r = grid.move(a)
                V = r + gamma * v[grid.current_state()]
                if V > best_value:
                    best_value = V
                    new_a = a
            policy[s] = new_a
            if new_a != old_a:
                is_policy_converged = False

    if is_policy_converged:
        break
        
print("Final values:")
print_values(v, grid)
print("\nFinal policy:")
print_policy(policy, grid)

Final values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|

Final policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |


## 演算法與flow chart    
#### 先隨機決定初始policy，在完成一次policy evaluation後，根據現有的V[s]來找到更加的policy(移動方向)，然後再去做policy evaluation與計算更加的移動policy，不斷循環直到無法找到更佳的policy。
![title](image01.png)
![title](PolicyIteration.jpg)

# 2. Value Iteration
## 第一步：將grid設定成standard狀態，再將可走動的格子reward全設成-0.1。    
## 並隨機選擇各格子要前進的方向(policy)

In [19]:
grid = negative_grid()
states = grid.all_states()
print("states:")
print(states)

print("\nrewards:")
print_values(grid.rewards, grid)

states:
{(0, 0), (1, 3), (2, 1), (2, 3), (1, 0), (0, 3), (0, 1), (1, 2), (2, 0), (2, 2), (0, 2)}

rewards:
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [20]:
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
print("policy:")
print_policy(policy, grid)

policy:
---------------------------
  R  |  U  |  D  |     |
---------------------------
  D  |     |  L  |     |
---------------------------
  L  |  U  |  R  |  L  |


In [24]:
#initialize V, gamma, policy, grid
v = {}
gamma = 0.9
small_enough = 1e-3
for s in states:
    v[s] = 0
    
while True:
    biggest_change = 0
    for s in states:
        old_v = v[s]
        if s in policy:
            best_v = float('-inf')
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                V = r + gamma * v[grid.current_state()]
                if V > best_v:
                    best_v = V
            v[s] = best_v
        biggest_change = max(biggest_change, np.abs(old_v-v[s]))
        
    if biggest_change < small_enough:
        break
        
        
for s in policy.keys():
    best_a = None
    best_value = float('-inf')
    # loop through all possible actions to find the best current action
    for a in ALL_POSSIBLE_ACTIONS:
        grid.set_state(s)
        r = grid.move(a)
        V = r + gamma * v[grid.current_state()]
        if V > best_value:
            best_value = V
            best_a = a
    policy[s] = best_a

# our goal here is to verify that we get the same answer as with policy iteration
print("values:")
print_values(v, grid)
print("\npolicy:")
print_policy(policy, grid)

values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |


## 演算法與flow chart
![title](image02.png)
![title](ValueIteration.jpg)